In [1]:
import tensorflow as tf
import numpy as np
tf.config.run_functions_eagerly(True)

from component.model import get_covid_classifier
from tensorflow import keras
from sklearn.utils import class_weight
from component.vocal_cords import get_vocal_cords_model
from utils.feature_extraction import  get_MFCCS

In [2]:
clf = get_covid_classifier(input_shape=(300, 200, 1),
                 vocal_cords_path='pretrained_weight/vocal_cords_v1.h5',
                 sentiment_path='pretrained_weight/sentiment_v1.h5',
                 tract_path=None)
clf.summary()


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 200, 1) 0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 300, 200, 1)  2           input_1[0][0]                    
__________________________________________________________________________________________________
distribution_lambda (Distributi multiple             0           dense[0][0]                      
__________________________________________________________________________________________________
model_4 (Functional)            (None, 10, 7, 2048)  23581440    distribution_lambda[0][0]        
____________________________________________________________________________________________

In [3]:
import random
class DataGenerator(keras.utils.Sequence):
    def __init__(self, X, y, batch_size):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.indexes = set(np.arange(len(X)))

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, _):
        indexes = random.sample(self.indexes, self.batch_size)
        X = []
        y = []
        for index in indexes:
            try:
                X.append(get_MFCCS(self.X[index]))
            except:
                continue
            y.append(self.y[index])
        return np.array(X), np.array(y, dtype=int)

In [4]:
import glob
X1 = glob.glob("merge_dataset/train/aug_p/*.wav")
X0 = glob.glob("merge_dataset/train/aug_n/*.wav")
X = X1 + X0
y = [1]*len(X1) + [0] * len(X0)
print(f"Found {len(X1)} labels positive")
print(f"Found {len(X0)} labels negative")

Found 2603 labels positive
Found 19465 labels negative


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=69, stratify=y)
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = {1 - i: x for i,x in enumerate(class_weights)}
print(f"Class weights: {class_weights}")

Class weights: {1: 0.5668507577703571, 0: 4.239673390970221}


d:\project\papyrus\venv\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1], y=[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
train_generator = DataGenerator(X_train, y_train, 8)
val_generator = DataGenerator(X_val, y_val, 8)

In [7]:
clf.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['acc'])
history = clf.fit(train_generator, steps_per_epoch=30, validation_steps=50, epochs=50, validation_data=val_generator, class_weight=class_weights,
                  callbacks=[tf.keras.callbacks.ModelCheckpoint('pretrained_weight/model_v1.h5', monitor="val_loss", save_weights_only=True, verbose=1, mode='min', save_best_only=True),
                             tf.keras.callbacks.TensorBoard(log_dir='logs')])
          # callbacks=[keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)])

d:\project\papyrus\venv\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/50
30/30 [==============================] - 34s 760ms/step - loss: 3.5890 - acc: 0.5743 - val_loss: 2.1703 - val_acc: 0.1250

d:\project\papyrus\venv\lib\site-packages\librosa\filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "


30/30 [==============================] - 20s 672ms/step - loss: 2.7354 - acc: 0.6727 - val_loss: 1.8657 - val_acc: 0.1275

In [8]:
from sklearn.metrics import classification_report
import tqdm
X1_test = glob.glob("merge_dataset/test/aug_p/*.wav")
X0_test = glob.glob("merge_dataset/test/aug_n/*.wav")
X_test = []
y_test = []
for audio_path in tqdm.tqdm(X1_test):
    try:
        X_test.append(get_MFCCS(audio_path))
        y_test.append(1)
    except:
        continue

for audio_path in tqdm.tqdm(X0_test):
    try:
        X_test.append(get_MFCCS(audio_path))
        y_test.append(0)
    except:
        continue
X_test = np.array(X_test)

100%|██████████| 3040/3040 [00:59<00:00, 51.00it/s]


In [10]:
clf.load_weights("pretrained_weight/model_v1.h5")
y_pred = clf.predict(X_test)

array([[0.10493679],
       [0.10493673],
       [0.10491524],
       [0.10493699],
       [0.10493591],
       [0.10489669],
       [0.10493606],
       [0.1049108 ],
       [0.10485178],
       [0.7940655 ]], dtype=float32)

In [16]:
y_pred_x=np.rint(y_pred)
print(classification_report(y_pred_x, y_test))

              precision    recall  f1-score   support

         0.0       0.99      0.88      0.93      3424
         1.0       0.00      0.05      0.01        42

    accuracy                           0.87      3466
   macro avg       0.50      0.46      0.47      3466
weighted avg       0.97      0.87      0.92      3466

